# Algoritmo de busca vetorial
Vamos transformar os resumos em embeddings para poder fazer uma busca vetorial em cima

In [24]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
df = pd.read_parquet('dados/articles2.parquet')
df

,title,title_alt,authors,advisor1,keywords,date,publisher,campus,citation,resumo,abstract,uri,pdf_uri
unique_id,,,,,,,,,,,,,
28320,Validação de modelos utilizando problemas inve...,None,"[Silva, Vinícius Vitorelli Generoso da]","Koroishi, Edson Hideki","[Problemas inversos (Equações diferenciais), V...",2015-02-11,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"SILVA, Vinícius Vitorelli Generoso da. Validaç...",Existe uma dificuldade desde os primórdios da ...,From the beginning there is difficulty in engi...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36735,Destinação do lodo de estação de tratamento de...,Disposal of water treatment plant sludge in wa...,"[Tamankievies, Daniela Fatima]","Bortoli, Marcelo","[Água - Estações de tratamento, Lodo, Testes d...",2025-03-27,Universidade Tecnológica Federal do Paraná,Francisco Beltrao,"TAMANKIEVIES, Daniela Fatima. Destinação do lo...",O lodo gerado em Estações de Tratamento de Águ...,The sludge generated in Water Treatment Plants...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36437,Gramep: método livre de alinhamento baseado no...,Gramep: an alignment-free method based on the ...,"[Zanon, Matheus Henrique Pimenta]","Lopes, Fabricio Martins","[Genômica, Aprendizado do computador, Classifi...",2025-03-27,Universidade Tecnológica Federal do Paraná,Cornelio Procopio,"ZANON, Matheus Henrique Pimenta. Gramep: métod...",Com o avanço na tecnologia de sequenciamento d...,"With advances in DNA sequencing technology, th...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
36652,Frações na perspectiva de medição e a abordage...,Fractions from the measurement perspective and...,"[Shiinoki, Vanessa Garcia]","Rocha, Zenaide de Fatima Dante Correia","[Matemática - Estudo e ensino, Frações, Mediçã...",2025-03-27,Universidade Tecnológica Federal do Paraná,Londrina,"SHIINOKI, Vanessa Garcia. Frações na perspecti...",O objetivo desta pesquisa foi analisar os proc...,The objective of this research was to analyze ...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
37105,Consciência fonológica na alfabetização: propo...,Phonological awareness in the literacy: a prop...,"[Rosa, Daniela Dala]","Denardi, Didie Ana Ceni","[Língua inglesa, Fonologia, Literacia, Ensino ...",2025-03-28,Universidade Tecnológica Federal do Paraná,Pato Branco,"ROSA, Daniela Dala. Consciência fonológica na ...","Esta pesquisa explicatória (Gil, 2021) versa s...","This exploratory research (Gil, 2021) deals wi...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,Vibração livre e flambagem de placas de Mindli...,Free-vibration and buckling analysis of Mindli...,"[Araujo, Leilson Joaquim]","Abdalla Filho, João Elias","[Vibração, Método dos elementos finitos, Flamb...",2021-05-28,Universidade Tecnológica Federal do Paraná,Curitiba,"ARAUJO, Leilson Joaquim. Vibração livre e flam...",O método dos elementos finitos (MEF) é um méto...,The finite element method (FEM) is an approxim...,http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
25542,Equilíbrio de fases experimental de hidratos d...,Experimental phase equilibria of carbon dioxid...,"[Vasconcelos, Luiz Fernando Santos de]","Marcelino Neto, Moises Alves","[Hidratos, Dióxido de carbono, Metano, Cloreto...",2021-05-28,Universidade Tecnológica Federal do Paraná,Curitiba,"VASCONCELOS, Luiz Fernando Santos de. Equilíbr...","Em certas condições termodinâmicas, as molécul...","Under certain thermodynamic conditions, water ...",http://repositorio.utfpr.edu.br/jspui/handle/1...,https://repositorio.utfpr.edu.br/jspui/bitstre...
26006,Influência das etapas de branqueamento e tempe...,Influence of bleaching steps 

In [3]:
from FlagEmbedding import BGEM3FlagModel
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
chunk_size = 4
df_chunks = np.array_split(df, math.ceil(len(df)/chunk_size))
for i, chunk in tqdm(enumerate(df_chunks), total=len(df_chunks)):
    embeddings = model.encode(list(chunk['resumo'].values), batch_size=chunk_size, max_length=8192, return_dense=True, return_sparse=False, return_colbert_vecs=False)['dense_vecs']
    embeddings_df = pd.concat([chunk.reset_index()['unique_id'], pd.DataFrame(embeddings)], axis=1)
    output_path='dados/embeddings.csv'
    embeddings_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

/home/guilherme/anaconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
  0%|                                       | 3/4445 [01:44<43:04:08, 34.91s/it]
